In [1]:
import time
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from sklearn import pipeline      # Pipeline
from sklearn import preprocessing # OrdinalEncoder, LabelEncoder
from sklearn import impute
from sklearn import compose
from sklearn import metrics   

from sklearn import set_config
set_config(display='diagram') # Useful for display the pipeline

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
#dataset_half_S = pd.read_csv('Datasets/dataset_halfSecondWindow.csv', index_col=[0])
dataset = pd.read_csv('Datasets/dataset_halfSecondWindow.csv')

In [4]:
dataset.head(10)

,Unnamed: 0,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gravity#std,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,android.sensor.gyroscope#max,android.sensor.gyroscope#std,android.sensor.gyroscope_uncalibrated#mean,android.sensor.gyroscope_uncalibrated#min,android.sensor.gyroscope_uncalibrated#max,android.sensor.gyroscope_uncalibrated#std,android.sensor.light#mean,android.sensor.light#min,android.sensor.light#max,android.sensor.light#std,android.sensor.linear_acceleration#mean,android.sensor.linear_acceleration#min,android.sensor.linear_acceleration#max,android.sensor.linear_acceleration#std,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field#std,android.sensor.magnetic_field_uncalibrated#mean,android.sensor.magnetic_field_uncalibrated#min,android.sensor.magnetic_field_uncalibrated#max,android.sensor.magnetic_field_uncalibrated#std,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.orientation#std,android.sensor.pressure#mean,android.sensor.pressure#min,android.sensor.pressure#max,android.sensor.pressure#std,android.sensor.proximity#mean,android.sensor.proximity#min,android.sensor.proximity#max,android.sensor.proximity#std,android.sensor.rotation_vector#mean,android.sensor.rotation_vector#min,android.sensor.rotation_vector#max,android.sensor.rotation_vector#std,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std,target,user
0,0,156661,3.0,NaN,62.0,10.053802,4.328630,14.419712,2.359507,0.629076,0.436463,0.762086,0.114439,9.806645,9.806636,9.806650,0.000004,3.268103,1.089049,6.149834,1.637808,3.278420,1.094471,6.163808,1.639430,251.500000,251.0,252.0,0.707107,3.955144,0.876975,12.568767,2.245528,26.294306,24.449745,27.040895,0.655650,38.660465,35.825271,48.540087,3.515975,69.212414,47.012974,109.841741,20.480671,1009.638535,1009.63620,1009.64087,0.003302,8.0,8.0,8.0,0.0,0.551880,0.389123,0.809349,0.137706,1809.0,1809.0,1809.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bus,U12
1,1,156662,4.0,NaN,62.0,10.687402,6.373194,16.329818,2.912315,0.577839,0.376494,0.654833,0.079211,9.806640,9.806629,9.806648,0.000005,3.388659,1.416119,5.525595,1.203786,3.401418,1.432180,5.535890,1.202950,173.333333,147.0,218.0,38.888730,4.679740,0.689140,10.320328,2.157321,26.295125,24.138973,28.049956,0.890534,61.175710,49.878653,71.257140,6.865865,147.052105,112.267878,178.057293,18.099960,1009.638510,1009.62915,1009.64453,0.008216,8.0,8.0,8.0,0.0,0.943359,0.817395,0.997289,0.046025,1809.0,1809.0,1809.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bus,U12
2,2,156663,5.0,NaN,62.0,11.816800,4.817410,16.854629,3.199759,0.651511,0.585485,0.713222,0.033566,9.806636,9.806620,9.806648,0.000006,2.458691,1.555355,3.359180,0.496747,2.452535,1.552447,3.345467,0.492861,136.333333,110.0,163.0,26.501572,4.248316,0.971063,9.106198,1.945371,27.048843,25.574597,28.146225,0.706742,72.826001,70.576129,73.689280,0.620963,138.203285,116.716235,162.190961,14.463783,1009.589363,1009.57715,1009.60720,0.015795,0.0,0.0,0.0,0.0,0.943879,0.892086,0.981119,0.026574,1809.0,1809.0,1809.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bus,U12
3,3,156664,6.0,NaN,62.0,9.401312,6.124588,12.230792,1.572354,0.713226,0.693072,0.748520,0.014745,9.806635,9.806622,9.806648,0.000006,1.275368,0.330836,1.996520,0.417501,1.271005,0.324460,1.993731,0.418559,34.666667,16.0,70.0,30.615900,2.420904,0.498379,5.206280,1.175246,25.878

In [5]:
dataset.user.unique().tolist() #all users

['U12',
 'U3',
 'U6',
 'U1',
 'U7',
 'U4',
 'U13',
 'U10',
 'U8',
 'U2',
 'U11',
 'U5',
 'U9']

In [6]:
for name, group in dataset.groupby('user'):
    print(name,len(group)) 

U1 25700
U10 5717
U11 1780
U12 9081
U13 957
U2 2897
U3 3522
U4 2434
U5 186
U6 3029
U7 4228
U8 2229
U9 825


In [7]:
dataset.target.unique().tolist() #all users

['Bus', 'Car', 'Still', 'Train', 'Walking']

In [8]:
dataset.target = dataset.target.replace({'Car':'Vehicle','Bus':'Vehicle'})

In [9]:
for name, group in dataset.groupby('target'):
    print(name,len(group)) 

Still 12516
Train 12517
Vehicle 25035
Walking 12517


In [10]:
# encode the target
#dataset.target = dataset.target.replace({'Still':0, 'Walking':1, 'Car':2, 'Bus':3, 'Train':4})

In [11]:
#dataset.columns

id
time
activityrecognition # 2
android.sensor.accelerometer #mmms //Measures the acceleration force in m/s2 that is applied to a device on all three physical axes (x, y, and z), including the force of gravityacceleration force
android.sensor.game_rotation_vector # mmms //orientation of the device
android.sensor.gravity #mmms //Measures the force of gravity in m/s2 that is applied to a device on all three physical axes (x, y, z)
android.sensor.gyroscope #mmms //Measures a device's rate of rotation in rad/s around each of the three physical axes (x, y, and z)
android.sensor.light #mmms //Measures the ambient light level (illumination) in lx
android.sensor.linear_acceleration #mmms // Measures the acceleration force in m/s2 that is applied to a device on all three physical axes (x, y, and z), excluding the force of gravity
android.sensor.magnetic_field #mmms //Measures the ambient geomagnetic field for all three physical axes (x, y, z) in μT
android.sensor.magnetic_field_uncalibrated #mmms
android.sensor.orientation #mmms
android.sensor.pressure #mmms //Measures the ambient air pressure in hPa or mbar.
android.sensor.proximity #mmms //Measures the proximity of an object in cm relative to the view screen of a device. This sensor is typically used to determine whether a handset is being held up to a person's ear.
android.sensor.rotation_vector #mmms //Measures the orientation of a device by providing the three elements of the device's rotation vector.
android.sensor.step_counter #mmms
sound #mmms
speed #mmms
target
user

targets : walking, car, still, train and bus. 
26% of data is annoted as walking, 25% as driving a car, 24% as standing still, 20% as being on train, and 5% as being on bus. 

In [12]:
#dataset.info()

In [13]:
dataset.describe()

,Unnamed: 0,id,time,activityrecognition#0,activityrecognition#1,android.sensor.accelerometer#mean,android.sensor.accelerometer#min,android.sensor.accelerometer#max,android.sensor.accelerometer#std,android.sensor.game_rotation_vector#mean,android.sensor.game_rotation_vector#min,android.sensor.game_rotation_vector#max,android.sensor.game_rotation_vector#std,android.sensor.gravity#mean,android.sensor.gravity#min,android.sensor.gravity#max,android.sensor.gravity#std,android.sensor.gyroscope#mean,android.sensor.gyroscope#min,android.sensor.gyroscope#max,android.sensor.gyroscope#std,android.sensor.gyroscope_uncalibrated#mean,android.sensor.gyroscope_uncalibrated#min,android.sensor.gyroscope_uncalibrated#max,android.sensor.gyroscope_uncalibrated#std,android.sensor.light#mean,android.sensor.light#min,android.sensor.light#max,android.sensor.light#std,android.sensor.linear_acceleration#mean,android.sensor.linear_acceleration#min,android.sensor.linear_acceleration#max,android.sensor.linear_acceleration#std,android.sensor.magnetic_field#mean,android.sensor.magnetic_field#min,android.sensor.magnetic_field#max,android.sensor.magnetic_field#std,android.sensor.magnetic_field_uncalibrated#mean,android.sensor.magnetic_field_uncalibrated#min,android.sensor.magnetic_field_uncalibrated#max,android.sensor.magnetic_field_uncalibrated#std,android.sensor.orientation#mean,android.sensor.orientation#min,android.sensor.orientation#max,android.sensor.orientation#std,android.sensor.pressure#mean,android.sensor.pressure#min,android.sensor.pressure#max,android.sensor.pressure#std,android.sensor.proximity#mean,android.sensor.proximity#min,android.sensor.proximity#max,android.sensor.proximity#std,android.sensor.rotation_vector#mean,android.sensor.rotation_vector#min,android.sensor.rotation_vector#max,android.sensor.rotation_vector#std,android.sensor.step_counter#mean,android.sensor.step_counter#min,android.sensor.step_counter#max,android.sensor.step_counter#std,sound#mean,sound#min,sound#max,sound#std,speed#mean,speed#min,speed#max,speed#std
count,62585.000000,62585.000000,62585.000000,0.0,62585.000000,62585.000000,62585.000000,62585.000000,62583.000000,54000.000000,54000.000000,54000.000000,51923.000000,57051.000000,57051.000000,57051.000000,5.023300e+04,54874.000000,54874.000000,54874.000000,54499.000000,54046.000000,54046.000000,54046.000000,53542.000000,46225.000000,46225.000000,46225.000000,12924.000000,57011.000000,57011.000000,57011.000000,50576.000000,57204.000000,57204.000000,57204.000000,54198.000000,53823.000000,53823.000000,53823.000000,50534.000000,57169.000000,57169.000000,57169.000000,54810.000000,2.037100e+04,2.037100e+04,2.037100e+04,8932.000000,12277.000000,12277.000000,12277.000000,2949.000000,54745.000000,54745.000000,54745.000000,52694.000000,15611.000000,15611.000000,15611.000000,887.000000,50927.000000,50927.000000,50927.000000,1474.000000,36665.000000,36665.000000,36665.000000,1691.000000
mean,31292.000000,111294.400256,1150.046097,NaN,88.238444,9.988878,9.445858,10.683369,0.861880,0.753394,0.751321,0.755145,0.004322,9.806542,9.806539,9.806545,7.133861e-05,0.325687,0.232439,0.435124,0.142522,0.324672,0.274031,0.384113,0.130444,1300.307463,1292.353364,1307.722299,40.431151,1.317694,1.202990,1.555978,0.508294,57.185426,56.938666,57.429738,0.782471,138.050851,137.859941,138.250527,1.226522,210.028928,207.937658,212.132650,5.627420,6.018476e+04,7.167994e+04,7.528767e+04,0.010155,5.363471,5.315468,5.411257,2.149151,0.764903,0.761870,0.767251,0.005218,10360.702261,10360.686631,10360.717891,0.716806,78.575667,78.575427,78.575917,0.339107,8.174253,8.174253,8.174253,0.020950
std,18066.877636,71786.050656,1260.411319,NaN,23.995054,1.341609,1.582800,3.271857,1.650861,0.254025,0.254559,0.254168,0.013626,0.001739,0.001746,0.001735,5.102546e-04,0.648738,0.510354,0.916648,0.332698,0.639724,0.563731,0.800216,0.303260,10149.350328,10129.382445,10182.233110,890.728298,2.169264,2.003144,3.539346,1.212473,128.059792,128.051756,128.073494,1.798792,72.106672

In [14]:
dataset.isnull().sum()

Unnamed: 0                                             0
id                                                     0
time                                                   0
activityrecognition#0                              62585
activityrecognition#1                                  0
android.sensor.accelerometer#mean                      0
android.sensor.accelerometer#min                       0
android.sensor.accelerometer#max                       0
android.sensor.accelerometer#std                       2
android.sensor.game_rotation_vector#mean            8585
android.sensor.game_rotation_vector#min             8585
android.sensor.game_rotation_vector#max             8585
android.sensor.game_rotation_vector#std            10662
android.sensor.gravity#mean                         5534
android.sensor.gravity#min                          5534
android.sensor.gravity#max                          5534
android.sensor.gravity#std                         12352
android.sensor.gyroscope#mean  

In [15]:
# replace anroid.sensor & #
dataset.columns = dataset.columns.str.replace('android.sensor.','').str.replace('#','_')
dataset.head()

<ipython-input-15-e92dca201964>:2: FutureWarning: The default value of regex will change from True to False in a future version.
  dataset.columns = dataset.columns.str.replace('android.sensor.','').str.replace('#','_')


,Unnamed: 0,id,time,activityrecognition_0,activityrecognition_1,accelerometer_mean,accelerometer_min,accelerometer_max,accelerometer_std,game_rotation_vector_mean,game_rotation_vector_min,game_rotation_vector_max,game_rotation_vector_std,gravity_mean,gravity_min,gravity_max,gravity_std,gyroscope_mean,gyroscope_min,gyroscope_max,gyroscope_std,gyroscope_uncalibrated_mean,gyroscope_uncalibrated_min,gyroscope_uncalibrated_max,gyroscope_uncalibrated_std,light_mean,light_min,light_max,light_std,linear_acceleration_mean,linear_acceleration_min,linear_acceleration_max,linear_acceleration_std,magnetic_field_mean,magnetic_field_min,magnetic_field_max,magnetic_field_std,magnetic_field_uncalibrated_mean,magnetic_field_uncalibrated_min,magnetic_field_uncalibrated_max,magnetic_field_uncalibrated_std,orientation_mean,orientation_min,orientation_max,orientation_std,pressure_mean,pressure_min,pressure_max,pressure_std,proximity_mean,proximity_min,proximity_max,proximity_std,rotation_vector_mean,rotation_vector_min,rotation_vector_max,rotation_vector_std,step_counter_mean,step_counter_min,step_counter_max,step_counter_std,sound_mean,sound_min,sound_max,sound_std,speed_mean,speed_min,speed_max,speed_std,target,user
0,0,156661,3.0,NaN,62.0,10.053802,4.328630,14.419712,2.359507,0.629076,0.436463,0.762086,0.114439,9.806645,9.806636,9.806650,0.000004,3.268103,1.089049,6.149834,1.637808,3.278420,1.094471,6.163808,1.639430,251.500000,251.0,252.0,0.707107,3.955144,0.876975,12.568767,2.245528,26.294306,24.449745,27.040895,0.655650,38.660465,35.825271,48.540087,3.515975,69.212414,47.012974,109.841741,20.480671,1009.638535,1009.63620,1009.64087,0.003302,8.0,8.0,8.0,0.0,0.551880,0.389123,0.809349,0.137706,1809.0,1809.0,1809.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vehicle,U12
1,1,156662,4.0,NaN,62.0,10.687402,6.373194,16.329818,2.912315,0.577839,0.376494,0.654833,0.079211,9.806640,9.806629,9.806648,0.000005,3.388659,1.416119,5.525595,1.203786,3.401418,1.432180,5.535890,1.202950,173.333333,147.0,218.0,38.888730,4.679740,0.689140,10.320328,2.157321,26.295125,24.138973,28.049956,0.890534,61.175710,49.878653,71.257140,6.865865,147.052105,112.267878,178.057293,18.099960,1009.638510,1009.62915,1009.64453,0.008216,8.0,8.0,8.0,0.0,0.943359,0.817395,0.997289,0.046025,1809.0,1809.0,1809.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vehicle,U12
2,2,156663,5.0,NaN,62.0,11.816800,4.817410,16.854629,3.199759,0.651511,0.585485,0.713222,0.033566,9.806636,9.806620,9.806648,0.000006,2.458691,1.555355,3.359180,0.496747,2.452535,1.552447,3.345467,0.492861,136.333333,110.0,163.0,26.501572,4.248316,0.971063,9.106198,1.945371,27.048843,25.574597,28.146225,0.706742,72.826001,70.576129,73.689280,0.620963,138.203285,116.716235,162.190961,14.463783,1009.589363,1009.57715,1009.60720,0.015795,0.0,0.0,0.0,0.0,0.943879,0.892086,0.981119,0.026574,1809.0,1809.0,1809.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vehicle,U12
3,3,156664,6.0,NaN,62.0,9.401312,6.124588,12.230792,1.572354,0.713226,0.693072,0.748520,0.014745,9.806635,9.806622,9.806648,0.000006,1.275368,0.330836,1.996520,0.417501,1.271005,0.324460,1.993731,0.418559,34.666667,16.0,70.0,30.615900,2.420904,0.498379,5.206280,1.175246,25.878591,25.585543,26.053984,0.116360,69.046004,65.059204,72.460886,2.365450,147.486429,124.566591,181.111339,20.478035,1009.552170,1009.54565,1009.56006,0.007302,0.0,0.0,0.0,0.0,0.911758,0.891509,0.938264,0.014528,1809.0,1809.0,1809.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vehicle,U12
4,4,156665,7.0,NaN,62.0,10.389851,5.313790,38.533453,5.426978,0.707375,0.684707,0.749263,0.018754,9.806633,9.806622,9.806642,0.000006,2.143492,0.556858,4.168938,0.803692,2.152290,0.565603,4.180464,0.804827,4.000000,4.0,4.0,0.000000,3.961614,0.538076,28.775562,4.356935,25.599663,25.244801,26.109195,0.239648,69.424097,64.957373,74.915753,3.585474,142.051738,116.199775,186.976676,23.746712,1009.559070,1009.54710,1009.57104,0.016928,0.0,0.0,0.0,0.0,0.926211,0.888596,0.972659,0.021917,1809.0,1809.0,1809.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vehicle

In [16]:
dataset['rv_gyro_mean'] = dataset['rotation_vector_mean'] * dataset['gyroscope_mean']
dataset['lin_speed_mean'] = dataset['linear_acceleration_mean'] * dataset['speed_mean']
dataset['rv_gyro_prox_mean'] = dataset['rotation_vector_mean'] * dataset['gyroscope_mean']* dataset['proximity_std']
dataset['lin_speed_prox_mean'] = dataset['linear_acceleration_mean'] * dataset['speed_mean'] *dataset['proximity_std']
dataset['rv_gyro__grv_mean'] = dataset['rotation_vector_mean'] * dataset['gyroscope_mean']* dataset['game_rotation_vector_mean']

In [17]:
dataset = dataset[(['accelerometer_min',
                    'accelerometer_max',
                    'accelerometer_std',
                    'gyroscope_mean',
                    'gyroscope_uncalibrated_mean',
                    'gyroscope_std',
                    'linear_acceleration_mean',
                    'rotation_vector_mean',
                    'rotation_vector_max',
                    'proximity_std',
                    'magnetic_field_uncalibrated_min',
                    'sound_min',
                    'sound_max',
                    'sound_mean',
                    
                    'rv_gyro_mean',
                    
                    'lin_speed_mean',
                    'rv_gyro_prox_mean',
                    'lin_speed_prox_mean',
                    'rv_gyro__grv_mean',
                    
                    'user',
                    'target'
                 ])]

In [18]:
dataset.columns

Index(['accelerometer_min', 'accelerometer_max', 'accelerometer_std',
       'gyroscope_mean', 'gyroscope_uncalibrated_mean', 'gyroscope_std',
       'linear_acceleration_mean', 'rotation_vector_mean',
       'rotation_vector_max', 'proximity_std',
       'magnetic_field_uncalibrated_min', 'sound_min', 'sound_max',
       'sound_mean', 'rv_gyro_mean', 'lin_speed_mean', 'rv_gyro_prox_mean',
       'lin_speed_prox_mean', 'rv_gyro__grv_mean', 'user', 'target'],
      dtype='object')

In [19]:
dataset.head()

,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_uncalibrated_mean,gyroscope_std,linear_acceleration_mean,rotation_vector_mean,rotation_vector_max,proximity_std,magnetic_field_uncalibrated_min,sound_min,sound_max,sound_mean,rv_gyro_mean,lin_speed_mean,rv_gyro_prox_mean,lin_speed_prox_mean,rv_gyro__grv_mean,user,target
0,4.328630,14.419712,2.359507,3.268103,3.278420,1.637808,3.955144,0.551880,0.809349,0.0,35.825271,NaN,NaN,NaN,1.803601,NaN,0.0,NaN,1.134602,U12,Vehicle
1,6.373194,16.329818,2.912315,3.388659,3.401418,1.203786,4.679740,0.943359,0.997289,0.0,49.878653,NaN,NaN,NaN,3.196721,NaN,0.0,NaN,1.847191,U12,Vehicle
2,4.817410,16.854629,3.199759,2.458691,2.452535,0.496747,4.248316,0.943879,0.981119,0.0,70.576129,NaN,NaN,NaN,2.320708,NaN,0.0,NaN,1.511967,U12,Vehicle
3,6.124588,12.230792,1.572354,1.275368,1.271005,0.417501,2.420904,0.911758,0.938264,0.0,65.059204,NaN,NaN,NaN,1.162827,NaN,0.0,NaN,0.829358,U12,Vehicle
4,5.313790,38.533453,5.426978,2.143492,2.152290,0.803692,3.961614,0.926211,0.972659,0.0,64.957373,NaN,NaN,NaN,1.985326,NaN,0.0,NaN,1.404371,U12,Vehicle


In [20]:
# split user = ['Luca' 'Pierpaolo' 'IvanHeibi']
dataset_test = dataset[(dataset['user'] == 'U2')|(dataset['user'] == 'U3')|(dataset['user'] == 'U4')]
dataset_train = dataset[(dataset['user'] != 'U2')&(dataset['user'] != 'U3')&(dataset['user'] != 'U4')]
print(len(dataset))
print(len(dataset_test))
print(len(dataset_train))

62585
8853
53732


In [21]:
dataset_train.sample(10)

,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_uncalibrated_mean,gyroscope_std,linear_acceleration_mean,rotation_vector_mean,rotation_vector_max,proximity_std,magnetic_field_uncalibrated_min,sound_min,sound_max,sound_mean,rv_gyro_mean,lin_speed_mean,rv_gyro_prox_mean,lin_speed_prox_mean,rv_gyro__grv_mean,user,target
36453,9.971860,9.971860,0.001856,0.008527,0.033882,0.001303,0.015708,0.971656,0.971669,NaN,142.344430,88.926082,88.926082,88.926082,0.008285,NaN,NaN,NaN,0.007961,U1,Still
34678,9.992321,10.006050,0.007005,0.005985,0.037771,0.000084,0.012910,0.970473,0.970473,NaN,140.721380,83.511396,83.511396,83.511396,0.005808,NaN,NaN,NaN,0.005580,U1,Still
33265,9.791321,9.842450,0.015296,0.001851,0.016742,0.000510,0.030623,0.054909,0.059589,NaN,50.575587,NaN,NaN,NaN,0.000102,0.000000,NaN,NaN,0.000018,U12,Still
46035,10.128246,10.128246,0.019481,0.019080,0.034972,0.001568,0.084719,0.991094,0.991097,NaN,127.504771,84.826944,84.826944,84.826944,0.018910,NaN,NaN,NaN,0.018775,U1,Train
36928,10.177718,10.177718,0.074836,0.049553,0.057927,0.019096,0.129366,0.996190,0.996190,NaN,106.212940,69.349202,69.349202,69.349202,0.049364,NaN,NaN,NaN,0.048698,U1,Still
48012,9.314365,9.807378,0.348613,0.057132,0.087086,0.028000,1.246304,0.911804,0.911804,NaN,177.524477,81.921507,81.921507,81.921507,0.052093,NaN,NaN,NaN,0.044880,U7,Train
56102,2.143003,22.405428,4.374211,1.992898,1.990616,0.774393,8.032250,0.803895,0.918197,0.0,80.107179,NaN,NaN,NaN,1.602081,8.771625,0.0,0.0,1.379823,U12,Walking
43001,10.004592,10.186732,0.128793,0.017590,0.038199,0.017791,0.134225,0.985850,0.985952,NaN,135.823106,89.762268,89.762268,89.762268,0.017342,NaN,NaN,NaN,0.017155,U1,Train
62269,7.959471,8.353082,0.278325,1.742858,0.292121,1.374515,5.510913,0.974199,0.974199,NaN,111.006876,89.660043,89.660043,89.660043,1.697891,5.510913,NaN,NaN,1.691410,U1,Walking
6271,9.891799,9.891799,0.560081,0.497548,0.135324,0.512305,2.711207,0.346706,0.346706,NaN,146.293535,89.751251,89.751251,89.751251,0.172503,4.744611,NaN,NaN,0.147249,U1,Vehicle


In [22]:
#dataset_train = dataset_train.fillna(0)

In [23]:
dataset_test.head()

,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_uncalibrated_mean,gyroscope_std,linear_acceleration_mean,rotation_vector_mean,rotation_vector_max,proximity_std,magnetic_field_uncalibrated_min,sound_min,sound_max,sound_mean,rv_gyro_mean,lin_speed_mean,rv_gyro_prox_mean,lin_speed_prox_mean,rv_gyro__grv_mean,user,target
33,9.764513,14.028126,1.283204,0.615399,0.778458,0.655962,1.721314,0.955202,0.955202,NaN,36.378866,77.064227,77.064227,77.064227,0.587830,2.050172,NaN,NaN,0.554815,U3,Walking
49,3.324387,14.608536,4.031835,0.597423,0.619693,0.167046,4.223926,0.934642,0.937574,NaN,87.072128,77.221125,77.221125,77.221125,0.558377,NaN,NaN,NaN,0.555901,U2,Walking
51,4.595131,17.636696,3.551271,NaN,NaN,NaN,2.791507,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,U4,Walking
54,8.712177,9.963264,0.361961,0.447802,0.334574,0.116564,0.958200,0.983561,0.989576,NaN,77.496043,74.178611,74.178611,74.178611,0.440440,0.000000,NaN,NaN,0.393798,U3,Vehicle
56,5.371977,15.088214,3.438894,0.550989,0.749501,0.250989,3.610910,0.802652,0.823187,NaN,72.877824,78.588379,80.072657,79.330518,0.442252,4.333092,NaN,NaN,0.441829,U2,Walking


In [22]:
# drop user in test & split
dataset_train = dataset_train.drop(['user'], axis = 1)
dataset_test = dataset_test.drop(['user'], axis = 1)

In [23]:
num_vars = ['accelerometer_min',
            'accelerometer_max',
            'accelerometer_std',
            'gyroscope_mean',
            'gyroscope_uncalibrated_mean',
            'gyroscope_std',
            
            'linear_acceleration_mean',
            
            'rotation_vector_mean',
            'rotation_vector_max',
            'proximity_std',
            'magnetic_field_uncalibrated_min',
            'sound_min',
            'sound_max',
            'sound_mean',
            
            
            'rv_gyro_mean',
            'lin_speed_mean',
            #'rv_gyro_prox_mean',
            #'lin_speed_prox_mean',
            #'rv_gyro__grv_mean',
            ]


In [25]:
# YOUR CODE HERE
from sklearn.impute import KNNImputer
from sklearn.preprocessing import RobustScaler
num_4_treeModels = pipeline.Pipeline(steps=[
  ('imputer', impute.KNNImputer(n_neighbors=3)),
  ('Normalizer',RobustScaler())
  
])

tree_prepro = compose.ColumnTransformer(transformers=[
    ('num', num_4_treeModels, num_vars)
], remainder='drop') # Drop other vars not specified in num_vars or cat_vars

tree_prepro

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputer',
                                                  KNNImputer(n_neighbors=3)),
                                                 ('Normalizer',
                                                  RobustScaler())]),
                                 ['accelerometer_min', 'accelerometer_max',
                                  'accelerometer_std', 'gyroscope_mean',
                                  'gyroscope_uncalibrated_mean',
                                  'gyroscope_std', 'linear_acceleration_mean',
                                  'rotation_vector_mean', 'rotation_vector_max',
                                  'proximity_std',
                                  'magnetic_field_uncalibrated_min',
                                  'sound_min', 'sound_max', 'sound_mean',
                                  'rv_gyro_mean', 'lin_speed_mean'])])

In [26]:


from sklearn.tree          import DecisionTreeClassifier
from sklearn.ensemble      import RandomForestClassifier
from sklearn.ensemble      import ExtraTreesClassifier
from sklearn.ensemble      import AdaBoostClassifier
from sklearn.ensemble      import GradientBoostingClassifier
from sklearn.experimental  import enable_hist_gradient_boosting # Necesary for HistGradientBoostingClassifier
from sklearn.ensemble      import HistGradientBoostingClassifier
from xgboost               import XGBClassifier
from lightgbm              import LGBMClassifier
from catboost              import CatBoostClassifier



In [27]:
tree_classifiers = {
  "Decision Tree": DecisionTreeClassifier(),
  "Extra Trees":ExtraTreesClassifier(),
  "Random Forest":RandomForestClassifier(),
  "AdaBoost":AdaBoostClassifier(),
  "Skl GBM":GradientBoostingClassifier(),
  "Skl HistGBM":GradientBoostingClassifier(),
  "XGBoost":XGBClassifier(),
  "LightGBM":LGBMClassifier(),
  "CatBoost":CatBoostClassifier()
}

tree_classifiers = {name: pipeline.make_pipeline(tree_prepro, model) for name, model in tree_classifiers.items()}



tree_classifiers["LightGBM"]

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('num',
                                                  Pipeline(steps=[('imputer',
                                                                   KNNImputer(n_neighbors=3)),
                                                                  ('Normalizer',
                                                                   RobustScaler())]),
                                                  ['accelerometer_min',
                                                   'accelerometer_max',
                                                   'accelerometer_std',
                                                   'gyroscope_mean',
                                                   'gyroscope_uncalibrated_mean',
                                                   'gyroscope_std',
                                                   'linear_acceleration_mean',
                                                   'rotation_vector_mean',
                                                   'rotation_vector_max',
                                                   'proximity_std',
                                                   'magnetic_field_uncalibrated_min',
                                                   'sound_min', 'sound_max',
                                                   'sound_mean', 'rv_gyro_mean',
                                                   'lin_speed_mean'])])),
                ('lgbmclassifier', LGBMClassifier())])

In [24]:
# split 
x_train = dataset_train.drop(['target'], axis=1)
y_train = dataset_train.target
x_test = dataset_test.drop(['target'], axis=1)
y_test = dataset_test.target

In [43]:
x_train.columns

Index(['accelerometer_min', 'accelerometer_max', 'accelerometer_std',
       'gyroscope_mean', 'gyroscope_uncalibrated_mean', 'gyroscope_std',
       'linear_acceleration_mean', 'rotation_vector_mean',
       'rotation_vector_max', 'proximity_std',
       'magnetic_field_uncalibrated_min', 'sound_min', 'sound_max',
       'sound_mean', 'rv_gyro_mean', 'lin_speed_mean', 'rv_gyro_prox_mean',
       'lin_speed_prox_mean', 'rv_gyro__grv_mean'],
      dtype='object')

In [44]:
from sklearn import model_selection
results = pd.DataFrame({'Model': [], 'Accuracy': [], 'Bal Acc.': [], 'Time': []})

x_train, x_val, y_train, y_val = model_selection.train_test_split(
    x_train, y_train,
    test_size=0.2,
    stratify = y_train,   
    random_state=37
)

for model_name, model in tree_classifiers.items():

    start_time = time.time()
    model.fit(x_train, y_train)
    total_time = time.time() - start_time
    
    pred = model.predict(x_test)  
    
    results = results.append({"Model":    model_name,
                              "Accuracy": metrics.accuracy_score(y_test, pred)*100,
                              "Bal Acc.": metrics.balanced_accuracy_score(y_test, pred)*100,
                              "Time":     total_time},
                              ignore_index=True)


results_ord = results.sort_values(by=['Accuracy'], ascending=False, ignore_index=True)
results_ord.index += 1 
results_ord.style.bar(subset=['Accuracy', 'Bal Acc.'], vmin=0, vmax=100, color='#5fba7d')

/opt/anaconda3/envs/ml/lib/python3.8/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


[13:05:36] WARNING: ../src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
Learning rate set to 0.084978
0:	learn: 0.9834906	total: 32.1ms	remaining: 32s
1:	learn: 0.8870294	total: 60ms	remaining: 29.9s
2:	learn: 0.8097971	total: 80.8ms	remaining: 26.8s
3:	learn: 0.7405161	total: 106ms	remaining: 26.3s
4:	learn: 0.6831162	total: 128ms	remaining: 25.6s
5:	learn: 0.6305207	total: 152ms	remaining: 25.1s
6:	learn: 0.5861480	total: 186ms	remaining: 26.4s
7:	learn: 0.5470433	total: 219ms	remaining: 27.1s
8:	learn: 0.5113575	total: 246ms	remaining: 27.1s
9:	learn: 0.4805860	total: 274ms	remaining: 27.1s
10:	learn: 0.4529285	total: 292ms	remaining: 26.2s
11:	learn: 0.4300497	total: 318ms	remaining: 26.2s
12:	learn: 0.4097438	total: 342ms	remaining: 25.9s
13:	learn: 0.3910465	total: 387ms	remaining: 27.3s
14:	learn: 0.3

157:	learn: 0.0751333	total: 4.52s	remaining: 24.1s
158:	learn: 0.0748585	total: 4.55s	remaining: 24.1s
159:	learn: 0.0744219	total: 4.56s	remaining: 24s
160:	learn: 0.0741749	total: 4.59s	remaining: 23.9s
161:	learn: 0.0739673	total: 4.62s	remaining: 23.9s
162:	learn: 0.0737907	total: 4.65s	remaining: 23.9s
163:	learn: 0.0735107	total: 4.67s	remaining: 23.8s
164:	learn: 0.0732032	total: 4.69s	remaining: 23.7s
165:	learn: 0.0731084	total: 4.72s	remaining: 23.7s
166:	learn: 0.0728579	total: 4.75s	remaining: 23.7s
167:	learn: 0.0726042	total: 4.78s	remaining: 23.7s
168:	learn: 0.0722540	total: 4.81s	remaining: 23.7s
169:	learn: 0.0720794	total: 4.84s	remaining: 23.6s
170:	learn: 0.0717967	total: 4.86s	remaining: 23.5s
171:	learn: 0.0713911	total: 4.88s	remaining: 23.5s
172:	learn: 0.0711419	total: 4.91s	remaining: 23.5s
173:	learn: 0.0706723	total: 4.93s	remaining: 23.4s
174:	learn: 0.0703587	total: 4.96s	remaining: 23.4s
175:	learn: 0.0701069	total: 4.98s	remaining: 23.3s
176:	learn: 0.

321:	learn: 0.0423054	total: 8.73s	remaining: 18.4s
322:	learn: 0.0419759	total: 8.74s	remaining: 18.3s
323:	learn: 0.0418472	total: 8.78s	remaining: 18.3s
324:	learn: 0.0417920	total: 8.8s	remaining: 18.3s
325:	learn: 0.0417559	total: 8.83s	remaining: 18.2s
326:	learn: 0.0416768	total: 8.85s	remaining: 18.2s
327:	learn: 0.0415430	total: 8.88s	remaining: 18.2s
328:	learn: 0.0414772	total: 8.9s	remaining: 18.2s
329:	learn: 0.0414233	total: 8.93s	remaining: 18.1s
330:	learn: 0.0413362	total: 8.97s	remaining: 18.1s
331:	learn: 0.0412188	total: 8.99s	remaining: 18.1s
332:	learn: 0.0410999	total: 9.02s	remaining: 18.1s
333:	learn: 0.0409781	total: 9.05s	remaining: 18s
334:	learn: 0.0409033	total: 9.07s	remaining: 18s
335:	learn: 0.0408699	total: 9.09s	remaining: 18s
336:	learn: 0.0408071	total: 9.12s	remaining: 17.9s
337:	learn: 0.0407375	total: 9.14s	remaining: 17.9s
338:	learn: 0.0406299	total: 9.17s	remaining: 17.9s
339:	learn: 0.0404538	total: 9.19s	remaining: 17.8s
340:	learn: 0.040231

486:	learn: 0.0276594	total: 12.7s	remaining: 13.4s
487:	learn: 0.0276488	total: 12.8s	remaining: 13.4s
488:	learn: 0.0276185	total: 12.8s	remaining: 13.4s
489:	learn: 0.0275715	total: 12.8s	remaining: 13.3s
490:	learn: 0.0274813	total: 12.8s	remaining: 13.3s
491:	learn: 0.0274555	total: 12.9s	remaining: 13.3s
492:	learn: 0.0274044	total: 12.9s	remaining: 13.2s
493:	learn: 0.0273686	total: 12.9s	remaining: 13.2s
494:	learn: 0.0272727	total: 12.9s	remaining: 13.2s
495:	learn: 0.0272041	total: 12.9s	remaining: 13.2s
496:	learn: 0.0271375	total: 13s	remaining: 13.1s
497:	learn: 0.0270904	total: 13s	remaining: 13.1s
498:	learn: 0.0270385	total: 13s	remaining: 13.1s
499:	learn: 0.0269771	total: 13s	remaining: 13s
500:	learn: 0.0268872	total: 13.1s	remaining: 13s
501:	learn: 0.0268154	total: 13.1s	remaining: 13s
502:	learn: 0.0267258	total: 13.1s	remaining: 12.9s
503:	learn: 0.0266668	total: 13.1s	remaining: 12.9s
504:	learn: 0.0265968	total: 13.2s	remaining: 12.9s
505:	learn: 0.0265335	tota

650:	learn: 0.0197062	total: 17.4s	remaining: 9.34s
651:	learn: 0.0196514	total: 17.4s	remaining: 9.31s
652:	learn: 0.0196261	total: 17.5s	remaining: 9.28s
653:	learn: 0.0195821	total: 17.5s	remaining: 9.26s
654:	learn: 0.0195229	total: 17.5s	remaining: 9.23s
655:	learn: 0.0195040	total: 17.6s	remaining: 9.21s
656:	learn: 0.0194856	total: 17.6s	remaining: 9.18s
657:	learn: 0.0194426	total: 17.6s	remaining: 9.15s
658:	learn: 0.0193701	total: 17.6s	remaining: 9.13s
659:	learn: 0.0192847	total: 17.7s	remaining: 9.1s
660:	learn: 0.0192512	total: 17.7s	remaining: 9.07s
661:	learn: 0.0192111	total: 17.7s	remaining: 9.04s
662:	learn: 0.0191695	total: 17.7s	remaining: 9.01s
663:	learn: 0.0191400	total: 17.8s	remaining: 8.98s
664:	learn: 0.0191269	total: 17.8s	remaining: 8.95s
665:	learn: 0.0191129	total: 17.8s	remaining: 8.92s
666:	learn: 0.0190863	total: 17.8s	remaining: 8.89s
667:	learn: 0.0190540	total: 17.8s	remaining: 8.86s
668:	learn: 0.0190275	total: 17.9s	remaining: 8.84s
669:	learn: 0

811:	learn: 0.0149773	total: 22.4s	remaining: 5.18s
812:	learn: 0.0149685	total: 22.4s	remaining: 5.16s
813:	learn: 0.0149496	total: 22.4s	remaining: 5.13s
814:	learn: 0.0149121	total: 22.5s	remaining: 5.1s
815:	learn: 0.0148903	total: 22.5s	remaining: 5.07s
816:	learn: 0.0148712	total: 22.5s	remaining: 5.04s
817:	learn: 0.0148558	total: 22.6s	remaining: 5.02s
818:	learn: 0.0148366	total: 22.6s	remaining: 4.99s
819:	learn: 0.0147800	total: 22.6s	remaining: 4.96s
820:	learn: 0.0147547	total: 22.6s	remaining: 4.93s
821:	learn: 0.0147249	total: 22.7s	remaining: 4.91s
822:	learn: 0.0147075	total: 22.7s	remaining: 4.88s
823:	learn: 0.0146935	total: 22.7s	remaining: 4.85s
824:	learn: 0.0146515	total: 22.7s	remaining: 4.82s
825:	learn: 0.0146404	total: 22.8s	remaining: 4.79s
826:	learn: 0.0146109	total: 22.8s	remaining: 4.77s
827:	learn: 0.0145759	total: 22.8s	remaining: 4.74s
828:	learn: 0.0145638	total: 22.9s	remaining: 4.72s
829:	learn: 0.0145211	total: 22.9s	remaining: 4.69s
830:	learn: 0

978:	learn: 0.0117497	total: 27.5s	remaining: 591ms
979:	learn: 0.0117399	total: 27.6s	remaining: 562ms
980:	learn: 0.0117317	total: 27.6s	remaining: 534ms
981:	learn: 0.0117226	total: 27.6s	remaining: 506ms
982:	learn: 0.0117112	total: 27.6s	remaining: 478ms
983:	learn: 0.0117006	total: 27.7s	remaining: 450ms
984:	learn: 0.0116920	total: 27.7s	remaining: 422ms
985:	learn: 0.0116828	total: 27.7s	remaining: 393ms
986:	learn: 0.0116678	total: 27.7s	remaining: 365ms
987:	learn: 0.0116531	total: 27.8s	remaining: 337ms
988:	learn: 0.0116446	total: 27.8s	remaining: 309ms
989:	learn: 0.0116378	total: 27.8s	remaining: 281ms
990:	learn: 0.0116009	total: 27.8s	remaining: 253ms
991:	learn: 0.0115769	total: 27.9s	remaining: 225ms
992:	learn: 0.0115663	total: 27.9s	remaining: 197ms
993:	learn: 0.0115599	total: 27.9s	remaining: 169ms
994:	learn: 0.0115462	total: 27.9s	remaining: 140ms
995:	learn: 0.0115292	total: 28s	remaining: 112ms
996:	learn: 0.0115114	total: 28.1s	remaining: 84.4ms
997:	learn: 0

,Model,Accuracy,Bal Acc.,Time
1,LightGBM,68.757539,70.008011,8.668414
2,CatBoost,68.275030,68.844519,33.678079
3,XGBoost,66.103739,69.983049,12.873961
4,Skl GBM,62.967431,66.420204,12.610167
5,Skl HistGBM,62.967431,66.420204,14.010336
6,Extra Trees,62.002413,65.430842,4.379770
7,AdaBoost,60.313631,65.079254,4.841080
8,Decision Tree,55.971049,63.673541,4.224314
9,Random Forest,55.971049,61.734206,10.787553


In [32]:
from sklearn.metrics import classification_report,confusion_matrix

print(metrics.accuracy_score(y_test, pred))
print(confusion_matrix(y_test,pred))
print(classification_report(y_test, pred))

0.6929854286682481
[[ 636   51  146   30]
 [   1  409   70    3]
 [  95  465 1360  206]
 [ 305  121 1225 3730]]
              precision    recall  f1-score   support

       Still       0.61      0.74      0.67       863
       Train       0.39      0.85      0.53       483
     Vehicle       0.49      0.64      0.55      2126
     Walking       0.94      0.69      0.80      5381

    accuracy                           0.69      8853
   macro avg       0.61      0.73      0.64      8853
weighted avg       0.77      0.69      0.71      8853



In [46]:
import pickle

In [28]:
model = tree_classifiers["AdaBoost"]
model.fit(x_train, y_train)
pred = model.predict(x_test) 

In [31]:
from sklearn.metrics import classification_report,confusion_matrix
print(confusion_matrix(y_test,pred))
print(classification_report(y_test, pred))

[[ 636   51  146   30]
 [   1  409   70    3]
 [  95  465 1360  206]
 [ 305  121 1225 3730]]
              precision    recall  f1-score   support

       Still       0.61      0.74      0.67       863
       Train       0.39      0.85      0.53       483
     Vehicle       0.49      0.64      0.55      2126
     Walking       0.94      0.69      0.80      5381

    accuracy                           0.69      8853
   macro avg       0.61      0.73      0.64      8853
weighted avg       0.77      0.69      0.71      8853



In [49]:
# save the model to disk
filename = 'final_model_vehicle.sav'
pickle.dump(model, open(filename, 'wb'))



In [50]:
# load the model from disk
loaded_model = pickle.load(open(filename, 'rb'))
result = loaded_model.score(x_test, y_test)
print(result)

0.6031363088057901


In [ ]:
### split by user

In [54]:
df_U3 = dataset[dataset.user.isin(['U3'])]

In [55]:
df_U3

,accelerometer_min,accelerometer_max,accelerometer_std,gyroscope_mean,gyroscope_uncalibrated_mean,gyroscope_std,linear_acceleration_mean,rotation_vector_mean,rotation_vector_max,proximity_std,magnetic_field_uncalibrated_min,sound_min,sound_max,sound_mean,rv_gyro_mean,lin_speed_mean,rv_gyro_prox_mean,lin_speed_prox_mean,rv_gyro__grv_mean,user,target
33,9.764513,14.028126,1.283204,0.615399,0.778458,0.655962,1.721314,0.955202,0.955202,NaN,36.378866,77.064227,77.064227,77.064227,0.587830,2.050172,NaN,NaN,0.554815,U3,Walking
54,8.712177,9.963264,0.361961,0.447802,0.334574,0.116564,0.958200,0.983561,0.989576,NaN,77.496043,74.178611,74.178611,74.178611,0.440440,0.000000,NaN,NaN,0.393798,U3,Vehicle
102,9.012243,9.187340,0.070605,0.069503,0.078626,0.048140,0.868719,0.787570,0.802811,NaN,91.951478,6.020600,6.020600,6.020600,0.054738,0.000000,NaN,NaN,0.049684,U3,Walking
121,8.246220,9.563967,0.451773,0.045041,0.038644,0.027367,0.958079,0.993876,0.996117,NaN,70.436446,72.559600,72.559600,72.559600,0.044765,7.473023,NaN,NaN,0.044658,U3,Vehicle
172,3.332163,17.743662,4.272022,1.734294,1.331566,1.400249,8.823918,0.985658,0.993053,NaN,90.118737,73.702289,73.702289,73.702289,1.709421,11.029897,NaN,NaN,1.694735,U3,Walking
176,7.296465,11.227957,1.147118,0.733852,0.480115,0.725273,3.605911,0.834938,0.921029,NaN,82.089224,16.901961,16.901961,16.901961,0.612721,0.000000,NaN,NaN,0.599210,U3,Walking
214,9.167899,12.326819,1.103488,0.473603,0.586040,0.169833,2.124048,0.771890,0.771890,NaN,62.035461,6.020600,6.020600,6.020600,0.365569,2.529847,NaN,NaN,0.311759,U3,Walking
219,9.470341,12.978275,1.329297,0.492967,0.450584,0.280371,1.011233,0.739975,0.763360,NaN,67.134252,19.084850,19.084850,19.084850,0.364783,1.204429,NaN,NaN,0.360176,U3,Walking
271,8.079353,10.333152,0.681599,1.275823,0.357030,0.989114,3.132773,0.784435,0.784435,NaN,83.756390,32.255677,32.255677,32.255677,1.000800,0.000000,NaN,NaN,0.967763,U3,Walking
304,3.610607,19.117971,5.344576,1.440491,1.240854,0.445474,4.703250,0.650959,0.668730,NaN,94.093226,78.515521,78.515521,78.515521,0.937701,6.078847,NaN,NaN,0.902893,U3,Walking
